## MIPs Adjustable Weight

In [1]:
import random
from ortools.linear_solver import pywraplp

In [2]:
def solve_student_project_assignment(students, projects, project_preferences, student_preferences, project_pref_weight, student_pref_weight):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Solver not found")
        return

    # Decision variables for project assignment
    x = {}
    for s in students:
        for p in projects:
            if students[s]['major'] in projects[p]['majors']:
                x[s, p] = solver.BoolVar(f'x[{s},{p}]')

    # Decision variables for student pair preferences
    y = {}
    for s1 in student_preferences:
        for s2 in student_preferences[s1]:
            y[s1, s2] = solver.BoolVar(f'y[{s1},{s2}]')

    # Hard constraints
    # Each student must be assigned 1 project
    for s in students:
        solver.Add(solver.Sum([x[s, p] for p in projects if (s, p) in x]) == 1)

    # Do not exceed maximum number of students in a major for each project
    for p in projects:
        for m in projects[p]['majors']:
            solver.Add(solver.Sum([x[s, p] for s in students if students[s]['major'] == m and (s, p) in x]) <= projects[p]['majors'][m])

    # Student-student pairing preferences constraints
    for s1 in student_preferences:
        for s2 in student_preferences[s1]:
            for p in projects:
                if (s1, p) in x and (s2, p) in x:
                    solver.Add(y[s1, s2] <= x[s1, p])
                    solver.Add(y[s1, s2] <= x[s2, p])

    # Soft constraints
    # Adjusting the project preferences weight based on preference order
    project_preferences_score = solver.Sum([project_pref_weight * (len(project_preferences[s]) - project_preferences[s].index(p)) * x[s, p]
                                           for s in project_preferences
                                           for p in project_preferences[s]
                                           if (s, p) in x])
    
    student_preferences_score = solver.Sum([student_pref_weight * y[s1, s2]
                                            for s1 in student_preferences
                                            for s2 in student_preferences[s1]])

    # Objective function
    solver.Maximize(project_preferences_score + student_preferences_score)

    # Solve the problem
    status = solver.Solve()

    # Output the solution
    solution = []
    if status == pywraplp.Solver.OPTIMAL:
        # print('Solution:')
        for s in students:
            for p in projects:
                if (s, p) in x and x[s, p].solution_value() > 0.5:
                    solution.append({'student': s, 'project': p})
                    # print(f'Student {s} is assigned to project {p}')
        return solution
    else:
        print('The problem does not have an optimal solution.')
        return None

In [20]:
# Example Usage
students = {
    'Alice': {'major': 'Computer Science'},
    'Bob': {'major': 'Engineering'},
    'Joe': {'major': 'Engineering'},
    'Ben': {'major': 'Engineering'},
    'Jared': {'major': 'Computer Science'},
    'Bella': {'major': 'Computer Science'},
    # Add other students
}

projects = {
    'Project1': {'majors': {'Computer Science': 2, 'Engineering': 2}},
    'Project2': {'majors': {'Computer Science': 2, 'Engineering': 2}},
    # Add other projects
}

project_preferences = {
    'Alice': ['Project1', 'Project2'],
    'Bob': ['Project2', 'Project1'],
    'Joe': ['Project2', 'Project1'],
    'Ben': ['Project2', 'Project1'],
    'Jared': ['Project1', 'Project2'],
    'Bella': ['Project1', 'Project2'],
    # Add other project preferences
}

student_preferences = {
    'Alice': ['Joe'],
    'Joe': ['Ben'],
    # Add other student preferences
}

solve_student_project_assignment(students, projects, project_preferences, student_preferences, project_pref_weight=1, student_pref_weight=1)

[{'student': 'Alice', 'project': 'Project1'},
 {'student': 'Bob', 'project': 'Project2'},
 {'student': 'Joe', 'project': 'Project1'},
 {'student': 'Ben', 'project': 'Project2'},
 {'student': 'Jared', 'project': 'Project1'},
 {'student': 'Bella', 'project': 'Project2'}]

## Import Data

In [4]:
import pandas as pd
import json

In [5]:
students_df = pd.read_csv('students.csv', index_col=0)
students_df = students_df.set_index('id')
students_df = students_df.rename(columns={'Name': 'name', 'Degree': 'degree', 'Other_desc': 'desc', 'Major': 'major'})
students_df['project_order'] = students_df['project_order'].apply(lambda x: x[1:-1].replace("'", "").replace(" ", "").split(','))
students_df

,name,degree,desc,secondary_skills,project_order,major
id,,,,,,
0,"Dechant, Bradley",Mechanical Engineering,Other,"['Additive manufacturing', 'Circuit design', '...","[F23-11, F23-14, F23-50, F23-09, F23-64, F23-6...",ME
1,"Drosselmeyer, Dillon",Mechanical Eng - Aerospace,Professional project management experience,"['Advance manufacturing courses, but not takin...","[F23-50, F23-51, F23-87, F23-88, F23-77, F23-7...",ME
3,"Crane, Michael",Mechanical Eng - Aerospace,NaN,"['Circuit design', 'Microcontroller interfacin...","[F23-63, F23-111, F23-75, F23-77, F23-73, F23-...",ME
4,"Elliott, Tanner",Mechanical Engineering,NaN,"['Additive manufacturing', 'Advance manufactur...","[F23-60, F23-72, F23-51, F23-116, F23-101, F23...",ME
5,"Apted, Michaelan",Mechanical Engineering,NaN,"['Ecology', 'Structural engineering']","[F23-111, F23-92, F23-75, F23-80, F23-112, F23...",ME
...,...,...,...,...,...,...
435,"Sarff, Christopher",EE-Antenn & Wirels Comm Spclty,Leadership,"['Circuit design', 'Construction engineering',...","[F23-64, F23-63, F23-89, F23-88, F23-17.1, F23...",EE
436,"Noel, Reece",Mechanical Engineering,NaN,"['Circuit design', 'Construction engineering',...","[F23-63, F23-64, F23-13, F23-61, F23-62, F23-6...",ME
437,"Araki, Shuta",Electrical Engineering,NaN,"['Automotive experience', 'Construction engine...","[F23-111, F23-17.2, F23-17.1, F23-52, F23-20, ...",EE


In [6]:
students = students_df[['major']].to_dict(orient='index')

In [10]:
students

{0: {'major': 'ME'},
 1: {'major': 'ME'},
 3: {'major': 'ME'},
 4: {'major': 'ME'},
 5: {'major': 'ME'},
 6: {'major': 'ME'},
 7: {'major': 'EE'},
 8: {'major': 'Env.'},
 9: {'major': 'EE'},
 10: {'major': 'ME'},
 11: {'major': 'EE'},
 12: {'major': 'ME'},
 13: {'major': 'ME'},
 14: {'major': 'ME'},
 15: {'major': 'ME'},
 16: {'major': 'ME'},
 17: {'major': 'Env.'},
 18: {'major': 'ME'},
 19: {'major': 'CE'},
 20: {'major': 'ME'},
 21: {'major': 'ME'},
 22: {'major': 'ME'},
 23: {'major': 'ME'},
 24: {'major': 'ME'},
 25: {'major': 'ME'},
 26: {'major': 'ME'},
 27: {'major': 'ME'},
 28: {'major': 'CE'},
 29: {'major': 'ME'},
 30: {'major': 'ME'},
 31: {'major': 'EE'},
 32: {'major': 'CE'},
 33: {'major': 'ME'},
 34: {'major': 'CE'},
 35: {'major': 'ME'},
 36: {'major': 'EE'},
 37: {'major': 'CE'},
 38: {'major': 'Env.'},
 39: {'major': 'ME'},
 40: {'major': 'EE'},
 41: {'major': 'CE'},
 42: {'major': 'ME'},
 43: {'major': 'ME'},
 44: {'major': 'ME'},
 45: {'major': 'Env.'},
 46: {'majo

In [12]:
project_preferences = students_df[['project_order']].to_dict(orient='dict')['project_order']

In [13]:
projects_df = pd.read_csv('projects.csv', index_col=0)
projects_df = projects_df.rename(columns={'ID': 'id', 'Project name': 'project_name', 'Sponsor': 'sponsor', 'majors_requested': 'majors'})
projects_df['majors'] = projects_df['majors'].apply(lambda x: json.loads(x))
projects_df = projects_df.set_index('id')
projects_df.head(3)

,project_name,sponsor,majors
id,,,
F23-01,AISC Steel Bridge - Team #1,AISC,"{'CE': 7, 'Env.': 0, 'EE': 0, 'ME': 0, 'DE': 0..."
F23-02,AISC Steel Bridge - Team #2,AISC,"{'CE': 7, 'Env.': 0, 'EE': 0, 'ME': 0, 'DE': 0..."
F23-03,ASCE Concrete Canoe,ASCE,"{'CE': 7, 'Env.': 0, 'EE': 0, 'ME': 0, 'DE': 0..."


In [14]:
projects = projects_df.to_dict(orient='index')

## Solve

In [15]:
solution = solve_student_project_assignment(students, projects, project_preferences, student_preferences, project_pref_weight=1, student_pref_weight=1)
solution = pd.DataFrame(solution)

In [16]:
student_merge = pd.merge(left=students_df, right=solution, left_index=True, right_on='student')
results = pd.merge(left=student_merge, right=projects_df, left_on='project', right_index=True).sort_values('student')

In [17]:
results[['student', 'name', 'degree', 'major', 'desc', 'secondary_skills', 'project_order', 'project', 'project_name', 'sponsor', 'majors']]

,student,name,degree,major,desc,secondary_skills,project_order,project,project_name,sponsor,majors
0,0,"Dechant, Bradley",Mechanical Engineering,ME,Other,"['Additive manufacturing', 'Circuit design', '...","[F23-11, F23-14, F23-50, F23-09, F23-64, F23-6...",F23-11,NASA Robotic Mining Competition - Lunabotics,NASA & Mines Space Resources,"{'CE': 0, 'Env.': 0, 'EE': 1, 'ME': 8, 'DE': 0..."
1,1,"Drosselmeyer, Dillon",Mechanical Eng - Aerospace,ME,Professional project management experience,"['Advance manufacturing courses, but not takin...","[F23-50, F23-51, F23-87, F23-88, F23-77, F23-7...",F23-50,3-Dimensional Oscillating Heat Pipes for Addit...,Ball Aerospace,"{'CE': 0, 'Env.': 0, 'EE': 0, 'ME': 6, 'DE': 0..."
2,3,"Crane, Michael",Mechanical Eng - Aerospace,ME,NaN,"['Circuit design', 'Microcontroller interfacin...","[F23-63, F23-111, F23-75, F23-77, F23-73, F23-...",F23-63,Restoration 1930 Ford Model A Flatbed Steak Truck,FRAPA,"{'CE': 0, 'Env.': 0, 'EE': 1, 'ME': 7, 'DE': 0..."
3,4,"Elliott, Tanner",Mechanical Engineering,ME,NaN,"['Additive manufacturing', 'Advance manufactur...","[F23-60, F23-72, F23-51, F23-116, F23-101, F23...",F23-72,Next Generation Telemark Binding,Bishop Telemark,"{'CE': 0, 'Env.': 0, 'EE': 0, 'ME': 4, 'DE': 1..."
4,5,"Apted, Michaelan",Mechanical Engineering,ME,NaN,"['Ecology', 'Structural engineering']","[F23-111, F23-92, F23-75, F23-80, F23-112, F23...",F23-80,"Skeletal Range of Motion Tracking, Analysis, a...",Vanderbilt University Medical Center,"{'CE': 0, 'Env.': 0, 'EE': 2, 'ME': 5, 'DE': 1..."
...,...,...,...,...,...,...,...,...,...,...,...
427,435,"Sarff, Christopher",EE-Antenn & Wirels Comm Spclty,EE,Leadership,"['Circuit design', 'Construction engineering',...","[F23-64, F23-63, F23-89, F23-88, F23-17.1, F23...",F23-64,ICE Swap to Electric in Sunbeam Classic Car (II),Thomas Eyers,"{'CE': 0, 'Env.': 2, 'EE': 3, 'ME': 3, 'DE': 0..."
428,436,"Noel, Reece",Mechanical Engineering,ME,NaN,"['Circuit design', 'Construction engineering',...","[F23-63, F23-64, F23-13, F23-61, F23-62, F23-6...",F23-64,ICE Swap to Electric in Sunbeam Classic Car (II),Thomas Eyers,"{'CE': 0, 'Env.': 2, 'EE': 3, 'ME': 3, 'DE': 0..."
429,437,"Araki, Shuta",Electrical Engineering,EE,NaN,"['Automotive experience', 'Construction engine...","[F23-111, F23-17.2, F23-17.1, F23-52, F23-20, ...",F23-17.2,Jefferson County Campus Solar Resources Evalua...,Jefferson County,"{'CE': 1, 'Env.': 0, 'EE': 2, 'ME': 2, 'DE': 0..."
430,438,"Gimadeyeva, Alexandra",Mechanical Eng - Energy,ME,NaN,"['Advance manufacturing courses, but not takin...","[F23-101, F23-102, F23-17.2, F23-17.1, F23-63,...",F23-101,Electroluminescence Camera Support Structure,BrightSpot Automation,"{'CE': 0, 'Env.': 0, 'EE': 2, 'ME': 3, 'DE': 0..."


In [18]:
# results.to_csv('mips_solution.csv')

## Analysis

In [80]:
def get_majors_by_project(lst):
    freq_dict = {}
    for el in lst:
        if el in freq_dict:
            freq_dict[el] += 1
        else:
            freq_dict[el] = 1
    res = list(freq_dict.items())
    res.sort(key=lambda x: x[1], reverse=True)
    return dict(res)

majors_by_project = results[['major', 'project', 'student']].groupby(['project']).agg({'major': ', '.join}).reset_index()
majors_by_project['major'] = majors_by_project['major'].apply(lambda x: x.split(', ')) # convert to list
majors_by_project['major'] = majors_by_project['major'].apply(lambda x: get_majors_by_project(x))
majors_by_project = majors_by_project.rename(columns={'major': 'majors_assigned'})
majors_by_project['students_assigned'] = majors_by_project['majors_assigned'].apply(lambda x: sum(x.values()))
majors_by_project = pd.merge(left=majors_by_project, right=projects_df, left_on='project', right_on='id')
majors_by_project['students_asked'] = majors_by_project['majors'].apply(lambda x: sum(x.values()))
majors_by_project.sort_values('students_assigned')

,project,majors_assigned,students_assigned,project_name,sponsor,majors,students_asked
2,F23- 115,{'EE': 1},1,The Ball Buddy,Sure Win LLC,"{'CE': 0, 'Env.': 0, 'EE': 1, 'ME': 5, 'DE': 0...",6
7,F23-06,{'Env.': 2},2,South Platte Renew Process Upgrades,RMWEA Competition,"{'CE': 2, 'Env.': 2, 'EE': 0, 'ME': 0, 'DE': 2...",6
0,F23 - 112,{'ME': 3},3,Temperature Controlled dog kennel,Daniel Caddy - Student Led,"{'CE': 0, 'Env.': 0, 'EE': 2, 'ME': 3, 'DE': 1...",6
26,F23-17.5,{'ME': 3},3,Campus Food Pantry Design Optimization,Estelle Cronmille - Student Led,"{'CE': 0, 'Env.': 0, 'EE': 0, 'ME': 3, 'DE': 1...",4
12,F23-100,{'ME': 3},3,Drying of mixed air/water stream in hydropower...,Flotation Pumping Company,"{'CE': 0, 'Env.': 1, 'EE': 0, 'ME': 3, 'DE': 2...",7
...,...,...,...,...,...,...,...
45,F23-60,"{'ME': 6, 'EE': 4, 'Other': 1}",11,Multi-Axis Racecar Simulator,ME Department,"{'CE': 0, 'Env.': 0, 'EE': 4, 'ME': 6, 'DE': 0...",11
6,F23-04,"{'ME': 6, 'CE': 5}",11,ASCE Timber Strong Competition,ASCE,"{'CE': 5, 'Env.': 0, 'EE': 0, 'ME': 8, 'DE': 0...",13
19,F23-13,"{'ME': 11, 'EE': 1}",12,Shell Eco-Marathon,Shell,"{'CE': 0, 'Env.': 0, 'EE': 1, 'ME': 11, 'DE': ...",12
46,F23-61,"{'ME': 12, 'EE': 4, 'Other': 1}",17,Quick Swap Modular EV Powertrain,ME Department,"{'CE': 0, 'Env.': 0, 'EE': 4, 'ME': 12, 'DE': ...",17
